In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report

In [2]:
cpu_data = pd.read_csv('./cpu_anomaly_data2.csv')
cpu_data['timestamp'] = pd.to_datetime(cpu_data['timestamp'])

In [3]:
cpu_data

,cpu_usage,timestamp,anomaly
0,21.756007,2023-12-04 22:20:00+00:00,0
1,22.302332,2023-12-04 22:25:00+00:00,0
2,34.959884,2023-12-04 22:30:00+00:00,0
3,29.625328,2023-12-04 22:35:00+00:00,0
4,21.144231,2023-12-04 22:40:00+00:00,0
...,...,...,...
9991,43.327759,2024-01-08 14:55:00+00:00,0
9992,36.268165,2024-01-08 15:00:00+00:00,0
9993,40.028437,2024-01-08 15:05:00+00:00,0
9994,44.154303,2024-01-08 15:10:00+00:00,1


In [4]:
cpu_data['cpu_usage'] = cpu_data['cpu_usage']/100
cpu_data

,cpu_usage,timestamp,anomaly
0,0.217560,2023-12-04 22:20:00+00:00,0
1,0.223023,2023-12-04 22:25:00+00:00,0
2,0.349599,2023-12-04 22:30:00+00:00,0
3,0.296253,2023-12-04 22:35:00+00:00,0
4,0.211442,2023-12-04 22:40:00+00:00,0
...,...,...,...
9991,0.433278,2024-01-08 14:55:00+00:00,0
9992,0.362682,2024-01-08 15:00:00+00:00,0
9993,0.400284,2024-01-08 15:05:00+00:00,0
9994,0.441543,2024-01-08 15:10:00+00:00,1


In [5]:
num_lags = 2
for i in range(1, num_lags+1):
    cpu_data['lag_{}'.format(i)] = cpu_data['cpu_usage'].shift(i)
  
cpu_data = cpu_data.dropna()

In [6]:
cpu_data

,cpu_usage,timestamp,anomaly,lag_1,lag_2
2,0.349599,2023-12-04 22:30:00+00:00,0,0.223023,0.217560
3,0.296253,2023-12-04 22:35:00+00:00,0,0.349599,0.223023
4,0.211442,2023-12-04 22:40:00+00:00,0,0.296253,0.349599
5,0.352088,2023-12-04 22:45:00+00:00,0,0.211442,0.296253
6,0.271253,2023-12-04 22:50:00+00:00,0,0.352088,0.211442
...,...,...,...,...,...
9991,0.433278,2024-01-08 14:55:00+00:00,0,0.424970,0.383699
9992,0.362682,2024-01-08 15:00:00+00:00,0,0.433278,0.424970
9993,0.400284,2024-01-08 15:05:00+00:00,0,0.362682,0.433278
9994,0.441543,2024-01-08 15:10:00+00:00,1,0.400284,0.362682


In [7]:
X = cpu_data[['cpu_usage', 'lag_1', 'lag_2']]
y = cpu_data['anomaly']

In [9]:
X_train = X[:int(0.7*len(X))]
X_test = X[int(0.7*len(X)):]
y_train = y[:int(0.7*len(y))]
y_test = y[int(0.7*len(y)):]

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

C:\Users\HP\AppData\Local\Temp\ipykernel_15180\2535592905.py:3: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_train = y[:int(0.7*len(y))]
C:\Users\HP\AppData\Local\Temp\ipykernel_15180\2535592905.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_test = y[int(0.7*len(y)):]


(3,)

In [10]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, input_shape=(3,), activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
219/219 [==============================] - 1s 3ms/step - loss: 0.2054 - accuracy: 0.9648 - val_loss: 0.0637 - val_accuracy: 0.9890
Epoch 2/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0707 - accuracy: 0.9871 - val_loss: 0.0594 - val_accuracy: 0.9890
Epoch 3/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0641 - accuracy: 0.9871 - val_loss: 0.0519 - val_accuracy: 0.9890
Epoch 4/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0549 - accuracy: 0.9871 - val_loss: 0.0416 - val_accuracy: 0.9890
Epoch 5/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0423 - accuracy: 0.9874 - val_loss: 0.0299 - val_accuracy: 0.9900
Epoch 6/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0303 - accuracy: 0.9898 - val_loss: 0.0208 - val_accuracy: 0.9937
Epoch 7/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0228 - accuracy: 0.9929 - val_loss: 0.0182 - val_accuracy: 0.9970
Epoch 

In [13]:
predict = model.predict(X_test)
predict = np.where(predict > 0.5, 1, 0)

94/94 [==============================] - 0s 1ms/step


In [15]:
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2966
           1       1.00      0.88      0.94        33

    accuracy                           1.00      2999
   macro avg       1.00      0.94      0.97      2999
weighted avg       1.00      1.00      1.00      2999



In [16]:
model.save('cpu_model_2_3.h5')